In [2]:
# hello world
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
import string
import re
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
sc = pyspark.SparkContext()

In [3]:
from pyspark.sql import *

In [4]:
spark = SparkSession.builder.appName("Projec3")\
        .config("spark.some.config.option","some-value")\
        .getOrCreate()

In [5]:
df = spark.read.load("/home/cse587/CSE587/Project3/train.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')

In [6]:
df.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



In [12]:
df.filter(df['genre'].isNull()).show()
 #formated correctly

+--------+----------+--------------------+-----+
|movie_id|movie_name|                plot|genre|
+--------+----------+--------------------+-----+
|20580938| Talentime|A music teacher, ...| null|
|33904402|  Deadroom|Four Conversation...| null|
+--------+----------+--------------------+-----+



In [13]:

#punc = string.punctuation
def removePunctuation(s):
    print("type of s: ",s)
    return re.sub(r'[^\w\s]',"",s)
#data = re.sub(r'[^\w\s]',"",data)

In [8]:
udf = UserDefinedFunction(lambda x: removePunctuation(x), StringType())
df = df.select(*[udf(column).alias("plot") if column == "plot" else column for column in df.columns])
#new_df = df.withColumn("plot",)
df.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hardwor...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventhday Advent...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|plot The film ope...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [9]:
genreMap = spark.read.load("/home/cse587/CSE587/Project3/mapping.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')
genreMap = genreMap.withColumnRenamed('_c0','id')
genreMap = genreMap.withColumnRenamed('0','genre')
genreMap.show()

#this converts a column into a list for use inside encodeGenre() 
#since you cant perform computations on a dataframe inside a UDF
genreList = genreMap.select("genre").rdd.flatMap(lambda x: x).collect()

+---+----------------+
| id|           genre|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [28]:

def encodeGenre(s):
    #s : the value of col "genre" at some row
    retVal=""
    #s = re.sub(r'[^\\'']',"",s)
    #print("s: ",s)
    #print("type of s: ",type(s))
    if(s == None):
        return retVal
    #convert from string to list
    s = s[1:-1].split(", ")
    #print("type of s: ",type(s))
    #print(s)
    
    #remove single quotes, regex wasnt working for some reason
    temp=[]
    for x in s:
        #s.remove(x)
        x=x[1:-1]
#         print(x)
#         index = genreMap["0"==x]
#         print(index)
        temp.append(x)
    s=temp
    #print(s)
    
    #create one hot encoding, 
    for genre in genreList:
        if genre in s:
            retVal=retVal+"1"
        else:
            retVal=retVal+"0"
##### apparetnly it aint kosher to perform df operations inside a udf... so this hour of work was for nothing
#     t = genreMap.withColumn("encoding",F.when(genreMap.genre.isin(s),1).otherwise(0))
#     #t.show()
#     retVal = t.select("encoding").rdd.flatMap(lambda x: x).collect()
#     retVal = ''.join(map(str,retVal))
    return retVal
    
    

In [11]:
udf2 = UserDefinedFunction(lambda x: encodeGenre(x), StringType())
df = df.select(*[udf2(column).alias("genre") if column == "genre" else column for column in df.columns])
df.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hardwor...|10000100000000000000|
|31186339|    The Hunger Games|The nation of Pan...|10001000001000000100|
|20663735|          Narasimham|Poovalli Induchoo...|10001000000000001000|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|01000000000000000000|
|  595909|   A Cry in the Dark|Seventhday Advent...|10000110000000000000|
| 5272176|            End Game|The president is ...|10011000001000000000|
| 1952976|          Dark Water|plot The film ope...|10010001000000000000|
|24225279|                Sing|The story begins ...|10000000000000000000|
| 2462689|       Meet John Doe|Infuriated at bei...|11100000100000000001|
|20532852|Destination Meatball|A line of people ...|00000000000011010000|
|15401493|    Husband for Hire|Lola  a

In [12]:
#install numpy if you havent already, otherwise one of the ml.featuer imports will throw an error cause it needs numpy
#%pip install numpy
import numpy
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [13]:
#taken from https://github.com/apache/spark/blob/master/examples/src/main/python/ml/tf_idf_example.py
tokenizer = Tokenizer(inputCol="plot", outputCol="words")
wordsData = tokenizer.transform(df)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(wordsData)

#this will do part 2
# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(featurizedData)
# rescaledData = idfModel.transform(featurizedData)


In [14]:
featurizedData.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|         rawFeatures|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hardwor...|10000100000000000000|[shlykov, a, hard...|(262144,[806,1275...|
|31186339|    The Hunger Games|The nation of Pan...|10001000001000000100|[the, nation, of,...|(262144,[991,1739...|
|20663735|          Narasimham|Poovalli Induchoo...|10001000000000001000|[poovalli, induch...|(262144,[119,571,...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|01000000000000000000|[the, lemon, drop...|(262144,[619,1998...|
|  595909|   A Cry in the Dark|Seventhday Advent...|10000110000000000000|[seventhday, adve...|(262144,[1147,191...|
| 5272176|            End Game|The president is ...|10011000001000000000

In [15]:
######NOT WORKING BELOW HERE, just trying some stuff out to create the term-document matrix

In [16]:
#text_split = df.select("plot").rdd.map(lambda line: line.split(" "))

In [17]:
#text_word = text_split.map(lambda word: (word,1))

In [18]:
#word_count = text_word.reduceByKey(lambda a, b: b+a)

In [19]:
#word_count.collect()

In [29]:
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import substring 
from pyspark.sql.types import *

new_feature_thing = featurizedData.drop('movie_id', 'movie_name', 'plot', 'words')

columns = new_feature_thing.columns
columns = columns[::-1]
new_feature_thing = new_feature_thing[columns]
new_feature_thing = new_feature_thing.withColumnRenamed('rawFeatures','features')
new_feature_thing = new_feature_thing.withColumnRenamed('genre','all_label')

x = new_feature_thing.withColumn("label", new_feature_thing.all_label.substr(1,1).cast(IntegerType()))
x = x.drop('all_label')
print(x.collect())

Py4JJavaError: An error occurred while calling o449.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 13, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 342, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/home/cse587/spark-2.4

In [24]:
x.filter(x['features'] != None ).show()

+--------+-----+
|features|label|
+--------+-----+
+--------+-----+



In [25]:
lr = LogisticRegression(maxIter = 10,regParam=0.3,elasticNetParam=0.8)
lr.fit(x)

Py4JJavaError: An error occurred while calling o195.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 342, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/home/cse587/spark-2.4.0-bin-hadoo